In [ ]:
pip install tensorflow


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# --------------------------
# 1. Our small textual dataset
# --------------------------
text = "I love machine learning and I love deep learning"

# --------------------------
# 2. Tokenize (convert words → numbers)
# --------------------------
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

# Convert text to sequence of numbers
input_seq = []
words = text.split()

for i in range(3, len(words)):
    seq = words[i-3:i+1]
    input_seq.append(tokenizer.texts_to_sequences([' '.join(seq)])[0])

input_seq = np.array(input_seq)

# Last word as output, first 3 words as input
X = input_seq[:, :-1]
y = input_seq[:, -1]

# --------------------------
# 3. Build LSTM model
# --------------------------
model = Sequential()
model.add(Embedding(total_words, 10, input_length=3))
model.add(LSTM(100))
model.add(Dense(total_words, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# --------------------------
# 4. Train the model
# --------------------------
model.fit(X, y, epochs=, verbose=0)

# --------------------------
# 5. Prediction function
# --------------------------
def predict_next_word(model, tokenizer, text):
    seq = tokenizer.texts_to_sequences([text])[0]
    seq = pad_sequences([seq], maxlen=3, padding='pre')
    pred = model.predict(seq, verbose=0)
    predicted_index = np.argmax(pred)

    for word, index in tokenizer.word_index.items():
        if index == predicted_index:
            return word

# --------------------------
# 6. Try predictions
# --------------------------
print("Next word for 'I love machine' →", predict_next_word(model, tokenizer, "I love machine"))
print("Next word for 'I love deep' →", predict_next_word(model, tokenizer, "I love deep"))

loss, accuracy = model.evaluate(X, y, verbose=0)
print("Training Accuracy:", accuracy)
print("Training Loss:", loss)

Next word for 'I love machine' → learning
Next word for 'I love deep' → learning
Training Accuracy: 1.0
Training Loss: 0.28363272547721863


In [ ]:
# ----------------------------
# 1. Import libraries
# ----------------------------
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# ----------------------------
# 2. Load IMDb dataset
# ----------------------------
vocab_size = 10000       # use top 10,000 most frequent words
max_length = 200         # each review will be trimmed/padded to 200 words

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocab_size)

# ----------------------------
# 3. Padding sequences
# ----------------------------
X_train = pad_sequences(X_train, maxlen=max_length, padding='post')
X_test = pad_sequences(X_test, maxlen=max_length, padding='post')

# ----------------------------
# 4. Build LSTM model
# ----------------------------
model = Sequential()

# Embedding Layer: word → vector
model.add(Embedding(input_dim=vocab_size, output_dim=128, input_length=max_length))

# LSTM Layer
model.add(LSTM(128))

# Output layer: 0 or 1
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

print(model.summary())

# ----------------------------
# 5. Train the model
# ----------------------------
model.fit(X_train, y_train, epochs=3, batch_size=128, validation_split=0.2)

# ----------------------------
# 6. Evaluate on test data
# ----------------------------
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", accuracy)

# ----------------------------
# 7. Predict sentiment for your own text
# ----------------------------

# function to convert text into tokens
word_index = imdb.get_word_index()

def encode_review(text):
    tokens = []
    for word in text.lower().split():
        if word in word_index and word_index[word] < vocab_size:
            tokens.append(word_index[word])
        else:
            tokens.append(2)  # unknown word index
    return pad_sequences([tokens], maxlen=max_length, padding='post')

# test your own review
sample_review = "The movie was absolutely amazing and I loved it"
sample = encode_review(sample_review)

prediction = model.predict(sample)[0][0]
print("\nReview:", sample_review)
print("Prediction Score:", prediction)

if prediction > 0.5:
    print("Sentiment: Positive 🙂")
else:
    print("Sentiment: Negative 🙁")


17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/3
157/157 ━━━━━━━━━━━━━━━━━━━━ 119s 740ms/step - accuracy: 0.5278 - loss: 0.6971 - val_accuracy: 0.5556 - val_loss: 0.6745
Epoch 2/3
157/157 ━━━━━━━━━━━━━━━━━━━━ 115s 730ms/step - accuracy: 0.6145 - loss: 0.6366 - val_accuracy: 0.7532 - val_loss: 0.5628
Epoch 3/3
157/157 ━━━━━━━━━━━━━━━━━━━━ 140s 723ms/step - accuracy: 0.8030 - loss: 0.4740 - val_accuracy: 0.5664 - val_loss: 0.6858
782/782 ━━━━━━━━━━━━━━━━━━━━ 74s 94ms/step - accuracy: 0.5595 - loss: 0.6909
Test Accuracy: 0.5626000165939331
1641221/1641221 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 199ms/step

Review: The movie was absolutely amazing and I loved it
Prediction Score: 0.63208604
Sentiment: Positive 🙂


In [ ]:
pip install librosa tensorflow numpy


In [ ]:
import librosa
import numpy as np

# Load any audio file (wav, mp3)
audio_path = "music.wav"  # put your music file here
signal, sr = librosa.load(audio_path, sr=22050)

# Extract MFCC features
mfcc = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=13)

print("MFCC shape:", mfcc.shape)


MFCC shape: (13, 256)


In [ ]:
sequence_length = 10

X = []
y = []

for i in range(mfcc.shape[1] - sequence_length):
    X.append(mfcc[:, i:i+sequence_length])
    y.append(mfcc[:, i+sequence_length])

X = np.array(X)
y = np.array(y)

# LSTM needs shape: (samples, timesteps, features)
X = X.transpose((0, 2, 1))   # (samples, timesteps=10, features=13)

print("X shape:", X.shape)
print("y shape:", y.shape)


X shape: (246, 10, 13)
y shape: (246, 13)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

model = Sequential()
model.add(LSTM(64, input_shape=(10, 13), return_sequences=False))
model.add(Dense(32, activation='relu'))
model.add(Dense(13))   # predict next MFCC frame

model.compile(optimizer='adam', loss='mse')

model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                   │ (None, 64)             │        19,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 13)             │           429 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 22,477 (87.80 KB)

 Trainable params: 22,477 (87.80 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.fit(X, y, epochs=30, batch_size=32)


Epoch 1/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 21216.5859
Epoch 2/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 21152.5645 
Epoch 3/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 20791.7246 
Epoch 4/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 21318.6562 
Epoch 5/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 20890.6836 
Epoch 6/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 20595.8730
Epoch 7/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 20801.1504 
Epoch 8/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 20626.9863 
Epoch 9/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 20259.2891 
Epoch 10/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 20046.7695 
Epoch 11/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 19488.5625 
Epoch 12/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 19570.9941 
Epoch 13/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 18599.3398 
Epoch 14/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 19347.7832 
Epoch 15/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/

In [ ]:
# take last 10 frames from MFCC
input_seq = mfcc[:, -10:].T.reshape(1, 10, 13)

predicted_frame = model.predict(input_seq)
print("Predicted MFCC frame:\n", predicted_frame)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 255ms/step
Predicted MFCC frame:
 [[-114.81386      71.75262      34.683777     -3.76051     -20.149118
   -27.977697    -23.130495    -14.459229     -5.7741394     0.24889475
     2.76202       1.9860954     0.30724174]]


In [ ]:
import os
import librosa
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# ---------------------------
# 1. Load MFCC from file
# ---------------------------
def extract_mfcc(file_path, max_len=200):
    signal, sr = librosa.load(file_path, sr=22050)

    mfcc = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=20)

    # Pad or cut to fixed length
    if mfcc.shape[1] < max_len:
        pad = max_len - mfcc.shape[1]
        mfcc = np.pad(mfcc, ((0,0),(0,pad)), mode='constant')
    else:
        mfcc = mfcc[:, :max_len]

    return mfcc.T   # shape → (time, features)



In [ ]:
import os
import librosa
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# ---------------------------
# 1. Extract MFCC Features
# ---------------------------
def extract_mfcc(file_path, max_len=200):
    # Load audio (MP3/WAV both supported)
    signal, sr = librosa.load(file_path, sr=22050)

    # Extract MFCC (20 features)
    mfcc = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=20)

    # Ensure fixed length
    if mfcc.shape[1] < max_len:
        pad = max_len - mfcc.shape[1]
        mfcc = np.pad(mfcc, ((0,0),(0,pad)), mode='constant')
    else:
        mfcc = mfcc[:, :max_len]

    return mfcc.T  # (time, features)

# ---------------------------
# 2. Load Dataset (valid=1, invalid=0)
# ---------------------------
X = []
y = []

base = "data"

for label, folder in enumerate(["invalid", "valid"]):
    folder_path = os.path.join(base, folder)

    for file in os.listdir(folder_path):
        if file.lower().endswith((".wav", ".mp3")):
            path = os.path.join(folder_path, file)
            mfcc = extract_mfcc(path)
            X.append(mfcc)
            y.append(label)

X = np.array(X)
y = np.array(y)

print("Dataset Loaded:", X.shape, y.shape)

# ---------------------------
# 3. Build LSTM Model
# ---------------------------
model = Sequential()
model.add(LSTM(128, input_shape=(200, 20)))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # binary classification

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

# ---------------------------
# 4. Train Model
# ---------------------------
model.fit(X, y, epochs=25, batch_size=4)

# ---------------------------
# 5. Save Model (optional)
# ---------------------------
model.save("singer_verifier.h5")

# ---------------------------
# 6. Prediction Function
# ---------------------------
def check_valid(file_path):
    mfcc = extract_mfcc(file_path)
    mfcc = np.expand_dims(mfcc, axis=0)  # shape: (1, time, features)

    pred = model.predict(mfcc)[0][0]

    if pred > 0.5:
        return f"✔ VALID: Same Singer (Score: {pred:.3f})"
    else:
        return f"❌ NOT VALID: Different Singer (Score: {pred:.3f})"

# ---------------------------
# 7. Test Your Audio
# ---------------------------
test_file = "test.mp3"   # or test.wav
print(check_valid(test_file))


Dataset Loaded: (3, 200, 20) (3,)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 128)            │        76,288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 84,609 (330.50 KB)

 Trainable params: 84,609 (330.50 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/25
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.6667 - loss: 0.5489
Epoch 2/25
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step - accuracy: 1.0000 - loss: 0.4078
Epoch 3/25
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step - accuracy: 1.0000 - loss: 0.3352
Epoch 4/25
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step - accuracy: 1.0000 - loss: 0.2463
Epoch 5/25
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step - accuracy: 1.0000 - loss: 0.1945
Epoch 6/25
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step - accuracy: 1.0000 - loss: 0.1538
Epoch 7/25
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step - accuracy: 1.0000 - loss: 0.1231
Epoch 8/25
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step - accuracy: 1.0000 - loss: 0.1005
Epoch 9/25
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step - accuracy: 1.0000 - loss: 0.0835
Epoch 10/25
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step - accuracy: 1.0000 - loss: 0.0703
Epoch 11/25
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step - accuracy: 1.0000 - loss: 0.0588
Epoch 12/25
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step - accuracy: 1.0000 - loss:

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 225ms/step
✔ VALID: Same Singer (Score: 0.986)
